In [1]:
import torch
from glob import glob
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path=''
imagePatches = glob(path+'/**/*.png', recursive=True)
print(len(imagePatches))
imagePatches[1]

In [5]:
import fnmatch
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

In [6]:
y = []
for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)

In [7]:
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y
print(images_df.iloc[0,:][0])
images_df.groupby("labels")["labels"].count()

In [9]:
images_df.to_csv("images_df.csv")

# Get some subdataset

In [ ]:
image_df=images_df.sample(frac=0.1)
print(image_df.iloc[0,:][0])
image_df.groupby("labels")["labels"].count()

In [ ]:
train, test = train_test_split(image_df, stratify=image_df.labels, test_size=0.05,random_state=10)
train, val = train_test_split(train, stratify=train.labels, test_size=0.2,random_state=10)
print("Train dataset :{}".format(len(train)))
print("val dataset :{}".format(len(val)))
print("test dataset :{}".format(len(test)))

# Model and dataloader

In [ ]:
## Parameters for model

# Hyper parameters
num_epochs = 10
num_classes = 2
batch_size = 100
learning_rate = 0.000001

# Device configuration
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        
        image = cv2.imread(img_path)
        image = cv2.resize(image, (50,50))
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
trans_train=transforms.Compose([transforms.ToPILImage(),
                                transforms.Pad(64,padding_mode='reflect'),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomRotation(20),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])
trans_valid=transforms.Compose([transforms.ToPILImage(),
                                transforms.Pad(64,padding_mode='reflect'),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])
trans_test=transforms.Compose([transforms.ToPILImage(),
                                transforms.Pad(64,padding_mode='reflect'),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])
dataset_train=MyDataset(df_data=train, transform=trans_train)
dataset_valid=MyDataset(df_data=val, transform=trans_valid)
dataset_test=MyDataset(df_data=test, transform=trans_test)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)
loader_test= DataLoader(dataset = dataset_test, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=2)
        self.conv2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=2)
        self.conv3=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=2)
        self.conv4=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=2)
        self.conv5=nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,padding=2)
#         self.conv6=nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3,padding=2)
        
        self.bn1=nn.BatchNorm2d(32)
        self.bn2=nn.BatchNorm2d(64)
        self.bn3=nn.BatchNorm2d(128)
        self.bn4=nn.BatchNorm2d(256)
        self.bn5=nn.BatchNorm2d(512)
#         self.bn6=nn.BatchNorm2d(1024)
        
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
        self.avg=nn.AvgPool2d(4)

        self.fc=nn.Linear(512*1*1,2) # final

    def forward(self,x):
        # relu,leaky_relu
        x=self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x=self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x=self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x=self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x=self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
#         x=self.pool(F.leaky_relu(self.bn6(self.conv6(x))))
        x=self.avg(x)
        
#         print(x.shape)
        x=x.view(-1,512*1*1)
        x=self.fc(x)
        return x

 

In [ ]:
model=MyCNN().to(device)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adamax(model.parameters(),lr=learning_rate)


In [ ]:
total_step = len(loader_train)
total_loss = []
total_acc=[]
all_loss=[]
for epoch in range(num_epochs):
    inter_loss = 0.0
    
    total=0
    correct=0
    
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        # print(images.shape)
        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward and optimizer

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        inter_loss += loss.item()
        all_loss.append(loss.item())
        
        _, predicted = torch.max(outputs.data, 1)
        total+= labels.size(0)
        correct+= (predicted == labels).sum().item()
        acc=((predicted == labels).sum().item()/labels.size(0))
        
        if(i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}] Loss:{:4f}'.format(epoch +
                  1, num_epochs, i+1, total_step, loss.item()))

    total_loss.append(inter_loss/total_step)
    total_acc.append(correct/total)
    

In [ ]:
total_acc

In [ ]:
total_loss

# result

In [ ]:
#定义两个数组
# Loss_list = []
# Accuracy_list = []

# Loss_list.append(train_loss / (len(train_dataset)))
# Accuracy_list.append(100 * train_acc / (len(train_dataset)))

x1 = range(0, 10)
x2 = range(0, 10)
y1 = total_acc
y2 = total_loss
plt.subplot(2, 1, 1)
plt.plot(x1, y1, 'o-')
plt.title('Train accuracy vs. epoches')
plt.ylabel('Train accuracy')
plt.subplot(2, 1, 2)
plt.plot(x2, y2, '.-')
plt.xlabel('Train loss vs. epoches')
plt.ylabel('Train loss')
plt.show()
plt.savefig("accuracy_loss.jpg")


In [ ]:
torch.save(model.state_dict(), "./code/H3.pth")

In [ ]:
model.eval()
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct=0
    total=0
    for images,labels in loader_valid:
        images=images.to(device)
        labels=labels.to(device)
        ouputs=model(images)
        _,predicted=torch.max(ouputs.data,1)
        total+=labels.size(0)

        correct+=(predicted==labels).sum().item()
        for t,p in zip(labels.view(-1),predicted.view(-1)):
              confusion_matrix[t.long(),p.long()]+=1

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [ ]:
confusion_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
classes = ['0','1']
confusion_matrix = np.array([(3335,458),(385,1095)],dtype=np.float64)

plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Oranges) 
plt.title('confusion_matrix')
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=-45)
plt.yticks(tick_marks, classes)

thresh = confusion_matrix.max() / 2.

iters = np.reshape([[[i,j] for j in range(2)] for i in range(2)],(confusion_matrix.size,2))
for i, j in iters:
    plt.text(j, i, format(confusion_matrix[i, j])) 

plt.ylabel('Real label')
plt.xlabel('Prediction')
plt.tight_layout()
plt.show()
